# Underlying and RiskFactor

Required Packages

In [1]:
import sys, os
import pandas as pd
pypricing_directory = os.path.expanduser('~/ArfimaTools/pypricing')
sys.path.insert(1,pypricing_directory)
from data.underlyings import Underlying
from risk.risk_factors import *

Obtaining sample data. We use natural gas data from the Iberian market.

In [2]:
data = pd.read_excel('example_data.xlsx', index_col=0)

# 1. Underlying Class

The Underlying class creates a base object that holds the information of an Underlying (a time series of values). It is initialized by providing a name, and then values are assigned using the set_value method. It also includes various basic methods, such as get_value, get_return, get_ewma, etc. The library can be found in pypricing\pricing\underlyings.py.


### 1.1. Initialize an instance

In [3]:
price_series = data[['Natural Gas']].rename(columns={'Natural Gas':'Price'})
underlying_eg = Underlying('Example_prices')

The values are defined using the method 

_.set_data(self, data)_

In [4]:
underlying_eg.set_data(price_series)

### 1.2. Some Methods
The Underlying class has various utility methods, such as  
- .get_dates(): Returns the dates.  
- .get_value(): Returns values. Dates must be provided as a mandatory parameter.  
- .get_returns(): Returns returns. Dates must be provided as a mandatory parameter.  
- .get_ewma_vol: Returns a volatility calculation using an EWMA. Dates must be provided as a mandatory parameter.  


#### get_dates
*get_dates(self)*  
.get_dates() returns a list of all the dates in the Underlying. Useful for combining with .get_value() and .get_returns().


In [ ]:
underlying_eg.get_dates()

#### get_value
*get_value(self, dates)*  
.get_value() simply returns the values. The only required parameter is a list of dates.


In [ ]:
underlying_eg.get_value(['2017-02-03', '2018-02-04'])

In [ ]:
underlying_eg.get_value(underlying_eg.get_dates()).plot()

#### get_return
*get_return(self, dates, percentage=False, annualized=False, raw=False)*  
.get_return() returns the returns. In addition to a list of dates, it has the following boolean parameters:  
- raw: True if we want absolute returns. False returns log returns (Default is False).  
- percentage: True if we want the value as a percentage over 100, False for value over one. Basically multiplies by 100 (Default is False).  
- annualized: True to get annualized returns (Default is False).  


In [ ]:
underlying_eg.get_return(underlying_eg.get_dates()).plot(title='Relative returns')

#### get_ewma_vol
*get_ewma_vol(self, dates, l=0.94, window=None, percentage=False, annualized=False)*  

It has the following new parameters:  
- l: Indicates the memory of the EWMA. Default is 0.94.  
- window: Indicates the window used, in number of indices. For example, if the data frequency is daily, 20 means a window of twenty days. Default is None (uses the entire available window).  


In [ ]:
underlying_eg.get_ewma_vol(underlying_eg.get_dates()).plot()

# 2. Riskfactor Class
The Riskfactor class is an extension of the Underlying objects, to which a method of calculating volatility is associated. Although the Riskfactor class has several common methods, we will always work with its subclasses, which associate a volatility method with the Underlying. These are always initialized by providing an Underlying object and performing a .fit(). Among others, we have the following subclasses:

- class PlainEWMARiskFactor(RiskFactor): Uses volatility based on an EWMA.  
- class EWMARiskFactor(PlainEWMARiskFactor): Uses volatility based on an EWMA, applying an ARIMA beforehand.  
- class GARCHRiskFactor(RiskFactor): Uses volatility based on a GARCH.  

The library can be found in \pypricing\risk\risk_factors.py


### 2.1 Common Methods of Riskfactor
The Riskfactor class has several common methods that can be used across all subclasses. Among them:

- **get_dates(self)**  
  Returns the dates of the Underlying.  

- **get_return(self, dates=None)**  
  Returns the return of the Underlying.  

- **get_vol(self, dates=None, annualized=False, percentage=False)**  
  Returns the volatility.  

- **get_white_noise_quantile(self, dates, quantile, window=None)**  
  Returns the specified quantile of the white noise (return/vol).  


### 2.2 PlainEWMARiskFactor Subclass
It is initialized with  

*def __init__(self, underlying, memory=0.94, window=250, raw=False, tenor=None):*  


In [10]:
plain_ewma_eg = PlainEWMARiskFactor(underlying_eg)

Then we must run a fit so that it calculates the volatility, specifying the start and end dates for which we want the information. By default, it uses all the available data.

*fit(self, beginning_date=None, end_date=None)*

In [11]:
plain_ewma_eg.fit()

Para obtener datos usamos los métodos comúnes:

In [ ]:
plain_ewma_eg.get_vol(plain_ewma_eg.get_dates()).plot()

Note that it is equivalent to the get_ewma_vol method of the Underlying (both graphs are identical).


### 2.3 EWMARiskFactor Subclass
It is initialized with  

*def __init__(self, underlying, memory=0.94, window=250, raw=False):*  


In [13]:
ewma_99_eg = EWMARiskFactor(underlying_eg, memory=0.99, window=250)

Then we must run a fit so that it calculates the volatility, specifying the start and end dates for which we want the information. By default, it uses all the available data. We can also indicate if we want to see the ARIMA details.

*fit(self, beginning_date=None, end_date=None, print_arima=False)*

In [ ]:
ewma_99_eg.fit()

In [ ]:
ewma_99_eg.get_vol(ewma_99_eg.get_dates()).plot()

### 2.4 GARCHRiskFactor Subclass
It is initialized with  

*def __init__(self, underlying, p=1, q=1):*  

p and q specify the parameters of the GARCH.  


In [16]:
garch_eg = GARCHRiskFactor(underlying_eg)

Then we must run a fit so that it calculates the volatility. We can specify whether we want to see the ARIMA details.

*fit(self, use_arima=False)*

In [ ]:
garch_eg.fit(use_arima=True)

In [ ]:
garch_eg.get_vol(garch_eg.get_dates()).plot()

## 3. drawdownsRiskFactor and drawdownsPortfolio
Although the `drawdownsRiskFactor` and `drawdownsPortfolio` classes are also subclasses of `EmpiricalRiskFactor`, they differ from the previous ones and deserve a separate section. Both classes are based on using drawdowns as a conservative source for price calculation. A drawdown is defined as the worst price change detected for our position over a time window. Therefore, it is defined depending on whether the position is long or short:

Long drawdowns are defined as:

$$
D_{\text{long}, t} := \min(p_{t+1}, \dots, p_{t+H}) - p_t
$$

Short drawdowns are defined as:

$$
D_{\text{short}, t} := \max(p_{t+1}, \dots, p_{t+H}) - p_t
$$

From here, there are two subclasses:
- **drawdownsRiskFactor**: a risk factor that specifies a method for calculating drawdowns.
- **drawdownsPortfolio**: manages the combination of the drawdowns from two `drawdownsRiskFactor`s.


### 2.4 drawdownsRiskFactor Subclass
It is initialized with  

*def __init__(self, underlying, raw=False, tenor=None):*  

_raw_ specifies whether drawdowns are calculated on absolute returns (raw=True) or relative returns (raw=False).  
_tenor_ is a string that specifies whether the contract is monthly ('M'), quarterly ('Q'), or yearly ('Y').  


In [19]:
drawdownriskfactor_eg = drawdownsRiskFactor(underlying_eg, tenor='M')

In [20]:
drawdownriskfactor_eg.fit()

In [ ]:
drawdownriskfactor_eg.get_drawdowns(drawdownriskfactor_eg.get_dates())